In [14]:
#-----------------------------
# Importing relevant libraries
#-----------------------------

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from IPython.display import display
from PIL import Image

import numpy as np
import csv
import matplotlib.pyplot as plt

In [4]:
#-----------------------------------------------
# Importing data from the CSV in relevant format
#-----------------------------------------------
with open("fer2013.csv") as f:  #change data file to fer2013.csv
    content = f.readlines()

lines = np.array(content)

num_of_instances = lines.size

x_train = []
y_train = []
x_test = []
y_test = []

for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")  
        val = img.split(" ")    
        pixels = np.array(val, 'float32')
        emotion = tf.keras.utils.to_categorical(emotion, 7)
    
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("",end="")

In [5]:
#data transformation for train and test sets
x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

x_train /= 255 #normalize inputs between [0, 1]
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

In [16]:
#------------------------------------------
# CNN MODEL which has 2 Convolution Layers
#------------------------------------------
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=3)

val_loss, val_acc = model.evaluate(x_test, y_test) 
print(val_loss) 
print(val_acc) 

Epoch 1/3
28709/28709 [==============================] - 181s 6ms/step - loss: 1.5715 - acc: 0.3909
Epoch 2/3
28709/28709 [==============================] - 182s 6ms/step - loss: 1.3626 - acc: 0.4839
Epoch 3/3
3589/3589 [==============================] - 8s 2ms/step
1.3075080347712176
0.5065477849066041


In [19]:
model.save('my_model.h5')

In [6]:
model = load_model('my_model.h5')

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
 
pred_list = []; actual_list = []
predictions = model.predict(x_test)
 
for i in predictions:
    pred_list.append(np.argmax(i))
 
for i in y_test:
    actual_list.append(np.argmax(i))
 
confusion_matrix(actual_list, pred_list)

array([[168,   1,  21,  92,  99,  14,  72],
       [ 16,   8,   7,   5,  16,   0,   4],
       [ 69,   0,  93,  77, 127,  48,  82],
       [ 37,   0,  13, 712,  56,  17,  60],
       [ 93,   0,  44,  92, 269,  15, 140],
       [ 26,   0,  23,  45,  22, 267,  32],
       [ 61,   1,  26,  90, 120,   8, 301]], dtype=int64)

In [25]:
#img = image.load_img("IMG.jpg", color_mode = "grayscale", target_size=(48, 48))

img = Image.open("IMG.jpg")
x = np.array(img)
x = np.expand_dims(x, axis = 0)

x /= 255

custom = model.predict(x)
emotion_analysis(custom[0])

x = np.array(x, 'float32')
x = x.reshape([48, 48]);

ValueError: Error when checking input: expected conv2d_14_input to have shape (48, 48, 1) but got array with shape (381, 214, 3)